# Fractional VQE Portfolio Optimization

This notebook implements a **fractional-weight formulation** of the portfolio optimization problem using a **Variational Quantum Eigensolver**.

### Contents

1. **Problem Setup**
   - Expected returns vector $μ$
   - Covariance matrix $Σ$
   - Risk-aversion parameter $λ$

2. **Quantum Ansatz**
   - Hardware-efficient $R_Y$ rotations (one per asset)
   - $⟨Z⟩$ expectation mapped to fractional weights

3. **Cost Function**
   - Mean–variance objective: $−μᵀw + λ wᵀΣw$
   - Soft constraints for $Σw=1$ and $w ≥ 0$

4. **Optimization**
   - PennyLane `AdamOptimizer` with autograd
   - Track cost convergence across iterations
    
5. **Results & Visualization**
   - Optimized weights
   - Bar chart and pie chart of allocation
   - Convergence plot

### Notes

- Ansatz is single-layer $R_Y$; extend with entanglers or depth for more expressivity
- Penalty scaling is data-driven
- Use different seeds or $λ$ values to test robustness

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from pennylane import draw_mpl

np.random.seed(0)

In [ ]:
# Problem setup
mu = np.array([0.10, 0.20, 0.15, 0.12], requires_grad=False)  # Expected returns
Sigma = np.array([
    [0.005, -0.010, 0.004, -0.002],
    [-0.010, 0.040, -0.002, 0.004],
    [0.004, -0.002, 0.023, 0.002],
    [-0.002, 0.004, 0.002, 0.018]
], requires_grad=False)

n_assets = len(mu)
lam = 0.5  # Risk aversion parameter

In [ ]:
dev = qml.device("default.qubit", wires=n_assets, shots=None)

@qml.qnode(dev, interface="autograd")
def qnode(thetas):
    # Hardware-efficient R_Y layer
    for i in range(n_assets):
        qml.RY(thetas[i], wires=i)

    # Return ⟨Z⟩ for each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(n_assets)]

def angles_to_weights(thetas):
    # From ⟨Z⟩ to probability of |1⟩
    z = qml.math.stack(qnode(thetas))
    w = (1.0 - z) * 0.5  # In [0,1]

    # Soft simplex projection via normalization
    s = qml.math.sum(w)
    return w / (s + 1e-12), z

def scaled_penalty(mu, Sigma):
    # Scale penalties to objective magnitude
    ret_scale = float(np.sum(np.abs(mu)))
    risk_scale = float(np.trace(Sigma))
    base = max(ret_scale, risk_scale, 1e-3)
    return 50.0 * base, 10.0 * base

def objective(thetas):
    w, _ = angles_to_weights(thetas)
    expected_return = qml.math.dot(mu, w)
    risk = qml.math.dot(w, qml.math.dot(Sigma, w))

    # Soft constraints
    p_sum, p_neg = scaled_penalty(mu, Sigma)
    c_sum = p_sum * (qml.math.sum(w) - 1.0) ** 2
    c_neg = p_neg * qml.math.sum(qml.math.maximum(-w, 0.0))  # Barrier if any w<0 (rare but safe)
    return -(expected_return) + lam * risk + c_sum + c_neg

# Draw the circuit
sample_thetas = np.linspace(0, np.pi, n_assets)
fig, ax = draw_mpl(qnode)(sample_thetas)
fig.suptitle("Fractional Ansatz Circuit (RY encoding)", fontsize=14)
plt.show()

In [ ]:
# Optimize
thetas = np.array(np.random.uniform(0, np.pi, n_assets), requires_grad=True)
opt = qml.AdamOptimizer(stepsize=0.3)
steps = 75
history = []

for t in range(steps):
    thetas, cost = opt.step_and_cost(objective, thetas)
    if (t+1) % 5 == 0:
        w, z = angles_to_weights(thetas)
        history.append(float(cost))
        print(f"step {t+1:03d}  cost={float(cost):.6f}  sum(w)={float(np.sum(w)):.4f}")

In [ ]:
# Convergence plot
plt.figure()
plt.plot(range(5, steps+1, 5), history)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.title("VQE convergence")
plt.grid(True)
plt.show()

In [ ]:
# Results
w_opt, _ = angles_to_weights(thetas)
w_opt = w_opt / (np.sum(w_opt) + 1e-12)

print("\nOptimized allocation")
for i, w in enumerate(w_opt):
    print(f"Asset {i+1}: {float(w)*100:.2f}%")

In [ ]:
# Bar chart
plt.figure()
plt.bar([f"Asset {i+1}" for i in range(n_assets)], w_opt)
plt.ylabel("Weight")
plt.ylim(0, 1)
plt.title("Fractional VQE allocation")
plt.grid(axis="y")
plt.show()

In [ ]:
# Pie chart
plt.figure(figsize=(6,6))
plt.pie(w_opt, labels=[f"Asset {i+1}" for i in range(n_assets)], autopct="%1.1f%%", startangle=140)
plt.title("Fractional VQE Pie")
plt.axis("equal")
plt.show()